# About: Moodle環境の削除--EC2インスタンスの削除

---

EC2インスタンスを削除します。

## パラメータの設定

EC2インスタンスを削除するのに必要となるパラメータを設定します。

### Ansibleのグループを指定する

削除対象のEC２インスタンスがあるAnsibleのグループ名を指定してください。

既存のグループ名を確認するために`group_vars`にあるディレクトリの一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にしてAnsibleのグループ名を次のセルに指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group = 

#### チェック

指定された `target_group` の値が適切なものかチェックします。

`target_group` に対応する設定ファイルが存在していることを確認します。

In [ ]:
from pathlib import Path

target_dir = Path('group_vars') / target_group
if not (target_dir.is_dir() and len(list(target_dir.glob("*"))) > 0):
    target_file = Path('group_vars') / f"{target_group}.yml"
    if target_file.is_file():
        target_dir.mkdir(exist_ok=True)
        target_file.rename(target_dir / "00-moodle.yml")
    else:
        raise RuntimeError(f"ERROR: not exists {target_group}")

### AWSの認証情報

EC2インスタンスの操作を行うためのAWSのアクセスキーとシークレットキーを入力します。

アクセスキーの値を入力してください。

In [ ]:
from getpass import getpass
aws_access_key = getpass()

シークレットキーを入力してください。

In [ ]:
aws_secret_key = getpass()

このNotebookで実行するコマンドから、AWSの認証情報を参照できるようにするため入力された情報を環境変数に設定します。

In [ ]:
import os

os.environ['AWS_ACCESS_KEY_ID'] = aws_access_key
os.environ['AWS_SECRET_ACCESS_KEY'] = aws_secret_key

### 削除対象となるインスタンスIDの指定

削除対象となるEC2インスタンスのIDを指定します。

削除候補となるインスタンスのIDリストを表示します。

In [ ]:
%run scripts/group.py
ids = load_group_var(target_group, "instance_ids")
for x in ids:
    print(x)

削除対象のインスタンスIDを指定してください。

In [ ]:
# (例)
# target_instance_id = 'i-xxxxxxxxxxxxxxxxx'

target_instance_id = 

## EC2インスタンスの削除

Moodle環境を構築したEC2インスタンスを削除します。

In [ ]:
region = load_group_var(target_group, "aws_region")
!.venv/bin/ansible localhost -c local -m amazon.aws.ec2_instance -a \
    'instance_ids={target_instance_id} region={region} state=absent'

## Ansible 設定ファイルの後始末

Ansibleに関する設定ファイルの後始末を行います。

group_vars に記録されているパラメータを無効にするために group_vars ファイルをリネームします。

In [ ]:
!mv group_vars/{target_group}  group_vars/.{target_group}.$(date +"%Y%m%d%H%M%S").bak

インベントリから対象となるグループを削除します。

In [ ]:
%run scripts/group.py
remove_group_from_inventory_yml(target_group)

# 削除前との差分を表示します。
!diff -u inventory.yml.bak inventory.yml || true